## Bulletines Cleaning

I used this notebook to clean raw data previously scraped containing information from the monthly bulletines published by the Italian Ministry of Justice.

In [1]:
import pandas as pd
import numpy as np
import re

### Basic cleaning

- Converting numbers into integers (note that the `.` in Italy is being used as a decimal separator)
- Removing `Totals` (we'll calculate them when needed)
- Fixing empty values to numeric (from NaN to 0)
- Converting to integer

In [2]:

# Read CSV with thousands separator specified
df = pd.read_csv("../outputs/raw/bulletines_raw.csv", thousands=".")

# Remove rows where "Regione di detenzione" is "Totale"
df = df[df["Regione di detenzione"] != "Totale"]

# Replace empty strings and similar with NaN
df.replace(["", " ", "NaN", "nan"], np.nan, inplace=True)

# Fill NaN values with 0
df.fillna(0, inplace=True)

# Convert columns to integer type
df['Detenuti presenti - stranieri'] = df['Detenuti presenti - stranieri'].astype(int)

# Remove dots from numeric strings in the specified column
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "")

# Show a random sample of 5 rows
df.sample(5)


,Regione di detenzione,Sigla Provincia,Istituto,Tipo istituto,Capienza Regolamentare,Detenuti presenti - totale,Detenuti presenti - donne,Detenuti presenti - stranieri,Ultimo aggiornamento,ID
3514,MARCHE,AP,FERMO,CR,41,54,0,20,2023-02-28,SST418777
3638,CAMPANIA,CE,"ARIENZO ""G. DE ANGELIS""",CC,58,77,0,8,2023-01-31,SST414676
1836,SARDEGNA,CA,ISILI -,CR,130,87,0,51,2023-11-30,SST449508
6711,LAZIO,RM,"ROMA ""R. CINOTTI"" REBIBBIA N.C.1",CC,1155,1309,0,446,2021-09-30,SST348572
2543,LOMBARDIA,BG,"BERGAMO ""Don Fausto RESMINI""",CC,319,526,32,256,2023-07-31,SST439073


In [3]:
df.dtypes

Regione di detenzione            object
Sigla Provincia                  object
Istituto                         object
Tipo istituto                    object
Capienza Regolamentare            int64
Detenuti presenti - totale       object
Detenuti presenti - donne        object
Detenuti presenti - stranieri     int64
Ultimo aggiornamento             object
ID                               object
dtype: object

In [4]:
# Step 1: Replace dots in the 'Detenuti presenti - totale' column, then convert to numeric
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "").astype(float)
# Step 2: Fill any remaining NaN values with 0
df['Detenuti presenti - totale'].fillna(0, inplace=True)
# Step 3: Convert the cleaned column to integers
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].astype(int)
df['Detenuti presenti - donne'] = df['Detenuti presenti - donne'].astype(int)


/tmp/ipykernel_3277/1983749719.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Detenuti presenti - totale'].fillna(0, inplace=True)


In [5]:
df.sort_values('Detenuti presenti - stranieri', ascending=False).head(15)

,Regione di detenzione,Sigla Provincia,Istituto,Tipo istituto,Capienza Regolamentare,Detenuti presenti - totale,Detenuti presenti - donne,Detenuti presenti - stranieri,Ultimo aggiornamento,ID
13387,LOMBARDIA,MI,"MILANO ""F. DI CATALDO"" SAN VITTORE",CC,748,1149,83,732,2024-10-31,SST1428284
13197,LOMBARDIA,MI,"MILANO ""F. DI CATALDO"" SAN VITTORE",CC,748,1149,83,732,2024-10-31,SST1428284
2010,PIEMONTE,TO,"TORINO ""G. LORUSSO - L. CUTUGNO"" LE VALLETTE",CC,1118,1496,126,719,2023-10-31,SST447996
1221,LOMBARDIA,MI,"MILANO ""F. DI CATALDO"" SAN VITTORE",CC,754,1146,85,716,2024-02-29,SST462936
1630,PIEMONTE,TO,"TORINO ""G. LORUSSO - L. CUTUGNO"" LE VALLETTE",CC,1118,1491,126,714,2023-12-31,SST454090
1820,PIEMONTE,TO,"TORINO ""G. LORUSSO - L. CUTUGNO"" LE VALLETTE",CC,1118,1480,125,714,2023-11-30,SST449508
651,LOMBARDIA,MI,"MILANO ""F. DI CATALDO"" SAN VITTORE",CC,754,1150,85,709,2024-05-31,SST1407300
1411,LOMBARDIA,MI,"MILANO ""F. DI CATALDO"" SAN VITTORE",CC,754,1158,81,708,2024-01-31,SST459023
300,PIEMONTE,TO,"TORINO ""G. LORUSSO - L. CUTUGNO"" LE VALLETTE",CC,1117,1494,135,706,2024-07-31,SST1415275
1440,PIEMONTE,TO,"TORINO ""G. LORUSSO - L. CUTUGNO"" LE VALLETTE",CC,1118,1444,128,698,2024-01-31,SST459023


In [6]:
df.dtypes

Regione di detenzione            object
Sigla Provincia                  object
Istituto                         object
Tipo istituto                    object
Capienza Regolamentare            int64
Detenuti presenti - totale        int64
Detenuti presenti - donne         int64
Detenuti presenti - stranieri     int64
Ultimo aggiornamento             object
ID                               object
dtype: object

In [7]:
# Convert the 'Ultimo aggiornamento' column to datetime format
df['Ultimo aggiornamento'] = pd.to_datetime(df['Ultimo aggiornamento'])


In [8]:
df.dtypes

Regione di detenzione                    object
Sigla Provincia                          object
Istituto                                 object
Tipo istituto                            object
Capienza Regolamentare                    int64
Detenuti presenti - totale                int64
Detenuti presenti - donne                 int64
Detenuti presenti - stranieri             int64
Ultimo aggiornamento             datetime64[ns]
ID                                       object
dtype: object

### Fixing names

A problem we are encountering are the many different names each detention center has been registered with over the years. Here below we use [thefuzz](https://github.com/seatgeek/thefuzz) to do an initial fuzzy matching, and then fix the remaining ones manually. As a reference, we'll use the information we have scraped in a different notebook about all the detention centers currently operating in Italy.

In [9]:
df['Istituto'] = df['Istituto'].str.strip()
df['Istituto'] = df['Istituto'].str.replace(r"\s*-", "", regex=True)  # Removes any whitespace followed by a dash


In [10]:
sorted(df['Istituto'].unique())

['AGRIGENTO "P. DI LORENZO"',
 'AGRIGENTO "PASQUALE DI LORENZO"',
 'ALBA "G. MONTALTO"',
 'ALBA "GIUSEPPE MONTALTO"',
 'ALESSANDRIA "G. CANTIELLO  S. GAETA"',
 'ALESSANDRIA "G. CANTIELLO S. GAETA"',
 'ALESSANDRIA "SAN MICHELE"',
 'ALGHERO "G. TOMASIELLO"',
 'ALGHERO "GIUSEPPE TOMASIELLO"',
 'ALTAMURA',
 'ANCONA',
 'ANCONA "BARCAGLIONE"',
 'ARBUS "IS ARENAS"',
 'AREZZO',
 'ARIANO IRPINO "P. CAMPANELLO"',
 'ARIANO IRPINO "PASQUALE CAMPANELLO"',
 'ARIENZO',
 'ARIENZO "G. DE ANGELIS"',
 'ARIENZO "GENNARO DE ANGELIS"',
 'ASCOLI PICENO',
 'ASTI',
 'AUGUSTA',
 'AVELLINO "A. GRAZIANO" BELLIZZI',
 'AVELLINO "ANTIMO GRAZIANO" BELLIZZI',
 'AVERSA "F. SAPORITO"',
 'AVEZZANO',
 'BARCELLONA POZZO DI GOTTO',
 'BARI "F. RUCCI"',
 'BARI "FRANCESCO RUCCI"',
 'BELLUNO',
 'BENEVENTO',
 'BENEVENTO "M. GAGLIONE"',
 'BERGAMO',
 'BERGAMO "Don Fausto RESMINI"',
 'BIELLA',
 'BOLLATE "II C.R."',
 'BOLOGNA "R. D\'AMATO"',
 'BOLOGNA "ROCCO D\'AMATO"',
 'BOLZANO',
 'BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCI

In [11]:
from thefuzz import process

In [12]:
# Import standard names
df_standard_names = pd.read_csv('../outputs/clean/institutes_info.csv')
standard_names = df_standard_names['nome_istituto'].tolist()
standard_names

['Reggio Calabria Arghillà',
 'Brescia Verziano',
 'Busto Arsizio',
 'Como',
 'Cremona',
 'Lecco',
 'Lodi',
 'Mantova',
 'Monza',
 'Pavia',
 'Sondrio',
 'Varese',
 'Voghera',
 'Catanzaro',
 'Crotone',
 'Palmi',
 'Paola',
 'Rossano',
 'Vibo Valentia',
 'Ariano Irpino',
 'Aversa',
 'Carinola',
 'Eboli',
 'Gorizia',
 'Padova',
 'Santa Maria Capua Vetere',
 'Pordenone',
 'Rovigo',
 'Ascoli Piceno',
 'Castelfranco Emilia',
 'Fermo',
 'Treviso',
 'Fossombrone',
 'Trieste',
 'Udine',
 'Pesaro',
 'Venezia Santa Maria Maggiore',
 'Venezia Giudecca',
 'Verona',
 'Avezzano',
 'Campobasso',
 'Alba',
 'Cassino',
 'Chieti',
 'Asti',
 'Civitavecchia Giuseppe Passerini',
 'Chiavari',
 'Frosinone',
 'Cuneo',
 'Trento',
 'Brissogne - Aosta',
 'Isernia',
 'Genova Pontedecimo',
 'Larino',
 'Imperia',
 'Ivrea',
 'Paliano',
 'La Spezia',
 'Rieti',
 'Roma Regina Coeli',
 'Saluzzo',
 'Roma Rebibbia',
 'Roma Rebibbia IIIª casa',
 'Roma Rebibbia femminile',
 'Verbania',
 'Padova Nuovo Complesso',
 'Sulmona',
 '

In [13]:
df_copy = df
df_copy.head(4)

,Regione di detenzione,Sigla Provincia,Istituto,Tipo istituto,Capienza Regolamentare,Detenuti presenti - totale,Detenuti presenti - donne,Detenuti presenti - stranieri,Ultimo aggiornamento,ID
0,ABRUZZO,AQ,AVEZZANO,CC,53,71,0,24,2024-08-31,SST1418233
1,ABRUZZO,AQ,L'AQUILA,CC,228,166,12,15,2024-08-31,SST1418233
2,ABRUZZO,AQ,SULMONA,CR,323,443,0,10,2024-08-31,SST1418233
3,ABRUZZO,CH,CHIETI,CC,79,140,36,35,2024-08-31,SST1418233


In [14]:
def standardize_name(name):
    match, score = process.extractOne(name, standard_names)
    return match if score >= 90 else name  # Adjust score threshold as needed
# Apply fuzzy matching to standardize names
df_copy['Istituto'] = df_copy['Istituto'].apply(standardize_name)

In [15]:
sorted(df_copy['Istituto'].unique())

['Agrigento',
 'Alba',
 'Alessandria Cantiello, Gaeta',
 'Alessandria San Michele',
 'Alghero',
 'Altamura',
 'Ancona Barcaglione',
 'Arbus',
 'Arezzo',
 'Ariano Irpino',
 'Arienzo',
 'Ascoli Piceno',
 'Asti',
 'Augusta',
 'Avellino',
 'Aversa',
 'Avezzano',
 'BOLZANO',
 'BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO',
 'Barcellona Pozzo di Gotto',
 'Bari',
 'Belluno',
 'Benevento',
 'Bergamo',
 'Biella',
 'Bollate',
 'Bologna',
 'Brescia Verziano',
 'Brindisi',
 'Brissogne - Aosta',
 'Busto Arsizio',
 'CAGLIARI "E.SCALAS"',
 'CAGLIARI "ETTORE SCALAS"',
 'CAMERINO',
 'CIVITAVECCHIA "N.C."',
 'Caltagirone',
 'Caltanissetta',
 'Campobasso',
 'Carinola',
 'Cassino',
 'Castelfranco Emilia',
 'Castelvetrano',
 'Castrovillari',
 'Catania Bicocca',
 'Catania Piazza Lanza',
 'Catanzaro',
 'Chiavari',
 'Chieti',
 'Civitavecchia Giuseppe Passerini',
 'Como',
 'Cosenza',
 'Cremona',
 'Crotone',
 'Cuneo',
 'Eboli',
 'Enna',
 "FORLI'",
 'Favignana',
 'Fermo',


Because the names that did not match are uppercase, we can identify them easily and proceed with a mapping.

In [16]:
sorted(df_copy[df_copy['Istituto'].str.isupper()]['Istituto'].unique())

['BOLZANO',
 'BRESCIA "N. FISCHIONE" CANTON MONBELLO',
 'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO',
 'CAGLIARI "E.SCALAS"',
 'CAGLIARI "ETTORE SCALAS"',
 'CAMERINO',
 'CIVITAVECCHIA "N.C."',
 "FORLI'",
 'MILANO "F. DI CATALDO" SAN VITTORE',
 'MILANO "FRANCESCO DI CATALDO" SAN VITTORE',
 'NAPOLI "G. SALVIA" POGGIOREALE',
 'NAPOLI "GIUSEPPE SALVIA" POGGIOREALE',
 'NAPOLI "P. MANDATO" SECONDIGLIANO',
 'NAPOLI "PASQUALE MANDATO" SECONDIGLIANO',
 'PALERMO "A. LORUSSO" PAGLIARELLI',
 'PALERMO "ANTONIO LORUSSO" PAGLIARELLI',
 'PALERMO "C. DI BONA" UCCIARDONE',
 'PALERMO "CALOGERO DI BONA" UCCIARDONE',
 'ROMA "G. STEFANINI" REBIBBIA FEMMINILE',
 'ROMA "GERMANA STEFANINI" REBIBBIA FEMMINILE',
 'ROMA "R. CINOTTI" REBIBBIA N.C.1',
 'ROMA "RAFFAELE CINOTTI" REBIBBIA N.C.1',
 'ROMA "REBIBBIA TERZA CASA"',
 'SAN REMO "N.C."']

In [17]:
institute_mapping = {
    'BRESCIA "N. FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'CAGLIARI "E.SCALAS"': 'Cagliari Uta',
    'CAGLIARI "ETTORE SCALAS"': 'Cagliari Uta',
    'CAMERINO': 'Camerino',
    'CIVITAVECCHIA "N.C."': 'Civitavecchia Nuovo Complesso',
    "FORLI'": 'Forlì',
    'MILANO "F. DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'MILANO "FRANCESCO DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'NAPOLI "G. SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "GIUSEPPE SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "P. MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'NAPOLI "PASQUALE MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'PALERMO "A. LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "ANTONIO LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "C. DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'PALERMO "CALOGERO DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'ROMA "G. STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "GERMANA STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "R. CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "RAFFAELE CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "REBIBBIA TERZA CASA"': 'Roma Rebibbia III Casa',
    'SAN REMO "N.C."': 'Sanremo',
}

df_copy['Istituto'] = df_copy['Istituto'].replace(institute_mapping)

# Check the updated unique values
df_copy['Istituto'].unique()

array(['Avezzano', "L'Aquila", 'Sulmona', 'Chieti', 'Lanciano', 'Vasto',
       'Pescara', 'Teramo', 'Matera', 'Melfi', 'Potenza', 'Castrovillari',
       'Cosenza', 'Paola', 'Rossano', 'Catanzaro', 'Crotone',
       'Laureana di Borrello', 'Locri', 'Palmi',
       'Reggio Calabria Arghillà', 'Reggio Calabria Giuseppe Panzera',
       'Vibo Valentia', 'Ariano Irpino', 'Avellino', 'Lauro',
       "Sant'Angelo dei Lombardi", 'Benevento', 'Arienzo', 'Aversa',
       'Carinola', 'Santa Maria Capua Vetere', 'Napoli Poggioreale',
       'Napoli Secondigliano', 'Pozzuoli', 'Eboli', 'Salerno',
       'Vallo della Lucania', 'Bologna', 'Ferrara', 'Forlì',
       'Castelfranco Emilia', 'Modena', 'Piacenza', 'Parma', 'Ravenna',
       'Reggio Emilia', 'Rimini', 'Gorizia', 'Pordenone', 'Trieste',
       'Tolmezzo', 'Udine', 'Cassino', 'Frosinone', 'Paliano', 'Latina',
       'Rieti', 'Civitavecchia Giuseppe Passerini',
       'Civitavecchia Nuovo Complesso', 'Roma Rebibbia Femminile',
       'Roma 

In [18]:
df_copy.sample(5)

,Regione di detenzione,Sigla Provincia,Istituto,Tipo istituto,Capienza Regolamentare,Detenuti presenti - totale,Detenuti presenti - donne,Detenuti presenti - stranieri,Ultimo aggiornamento,ID
10264,ABRUZZO,CH,Lanciano,CC,231,257,0,12,2020-02-29,SST250612
2920,LIGURIA,IM,Imperia,CC,53,73,0,43,2023-05-31,SST431165
425,EMILIA ROMAGNA,RA,Ravenna,CC,49,78,0,43,2024-06-30,SST1410601
5650,SICILIA,CT,Catania Bicocca,CC,136,210,0,19,2022-03-31,SST374094
4338,TOSCANA,FI,Firenze Mario Gozzini,CC,92,91,0,53,2022-10-31,SST401327


In [19]:
# # remove duplicate data from Dec 2021
# df_copy = df_copy[df_copy['ID'] != 'SST360932']

In [20]:
df_copy = df_copy.sort_values(['Ultimo aggiornamento'], ascending=False)
df_copy.tail()

,Regione di detenzione,Sigla Provincia,Istituto,Tipo istituto,Capienza Regolamentare,Detenuti presenti - totale,Detenuti presenti - donne,Detenuti presenti - stranieri,Ultimo aggiornamento,ID
12858,PUGLIA,LE,Lecce,CC,610,1059,76,168,2019-01-31,SST168760
12857,PUGLIA,FG,San Severo,CC,62,80,0,23,2019-01-31,SST168760
12856,PUGLIA,FG,Lucera,CC,137,160,0,38,2019-01-31,SST168760
12855,PUGLIA,FG,Foggia,CC,365,600,27,85,2019-01-31,SST168760
12923,VENETO,VI,Vicenza,CC,286,322,0,189,2019-01-31,SST168760


In [21]:
df_copy.to_csv('../outputs/clean/bulletines.csv', index=False, encoding="UTF-8-sig")